In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention, Add
from sklearn.metrics import classification_report

embed_dim = 64     # 每个字符转成64维向量
num_heads = 2      # 注意力头数
ff_dim = 64        # 前馈层维度
max_len = 200

data=pd.read_csv('E:/作业/杂物/数据集/111_fixed.csv',encoding='ISO-8859-1')
data = data.dropna(subset=['url', 'label'])
data = data[data['url'].astype(str).str.strip() != '']
x=data['url']
y=data['label'] #加载数据

x = data['url'].astype(str)
tokenizer = Tokenizer(char_level=True, lower=True)  # lower=True 小写处理统一格式
tokenizer.fit_on_texts(urls)  # 学习所有字符的编号映射
vocab_size = len(tokenizer.word_index) + 1  # Transformer构建时需要知道有多少个字符token，+1 是因为 index 从 1 开始
sequences = tokenizer.texts_to_sequences(urls)  # 每个网址现在变成数字列表
x_pad = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post') #Transformer要求每一个样本的输入维度必须完全一致！

inputs = Input(shape=(max_len,))

# 词嵌入层（Embedding）
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)

# 多头注意力层
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
attention_output = Add()([embedding_layer, attention_output])
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# 前馈网络
ffn = Dense(ff_dim, activation='relu')(attention_output)
ffn = Dense(embed_dim)(ffn)
ffn_output = Add()([attention_output, ffn])
ffn_output = LayerNormalization(epsilon=1e-6)(ffn_output)

# 池化 + 输出层
x = GlobalAveragePooling1D()(ffn_output)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='sigmoid')(x)

# 构建模型
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

data2=pd.read_csv('E:/作业/杂物/数据集/balanced_test_set.csv', encoding='gb18030')
x1=data2['url'].astype(str)
y1=data2['label'] #读取测试数据

model.fit(x_pad, y, epochs=5, batch_size=128)


sequences_test = tokenizer.texts_to_sequences(x1)
x_pad_test = pad_sequences(sequences_test, maxlen=max_len, padding='post', truncating='post')

y_predict = model.predict(x_pad_test)
y_pred = (y_predict > 0.5).astype(int)
print(classification_report(y1.values, y_pred))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 200, 64)   │     10,944 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 200, 64)   │     33,216 │ embedding_1[0][0… │
│ (MultiHeadAttentio… │                   │            │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 200, 64)   │          0 │ embedding_1[0][0… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 200, 64)   │        128 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 200, 64)   │      4,160 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 200, 64)   │      4,160 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 200, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 200, 64)   │        128 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         65 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 52,801 (206.25 KB)

 Trainable params: 52,801 (206.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 64ms/step - accuracy: 0.7435 - loss: 0.5109
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 64ms/step - accuracy: 0.8244 - loss: 0.3934
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 65ms/step - accuracy: 0.8550 - loss: 0.3425
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 122s 65ms/step - accuracy: 0.8636 - loss: 0.3226
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 65ms/step - accuracy: 0.8693 - loss: 0.3088
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     50000
           1       0.46      0.85      0.60     50000

    accuracy                           0.43    100000
   macro avg       0.23      0.43      0.30    100000
weighted avg       0.23      0.43      0.30    100000

